In [1]:
from bs4 import BeautifulSoup as bs              
import urllib.request as req                     
import pandas as pd                              
import time
import gc

In [2]:
def ready_to_scrap_page_main(url):
    _ = req.Request(url)
    HTML = req.urlopen(_)
    page = bs(HTML.read(), 'html.parser')
    resttable = page.find("ul", {"class":"list-restaurants"})
    rests = resttable.findAll("div",{"class":"list-restaurant-item"})
    return rests

In [3]:
def ready_to_scrap_page_detail(url):
    _ = req.Request(url)
    HTML = req.urlopen(_)
    page = bs(HTML.read(), 'html.parser')
    infotable = page.find('section', {'class':'restaurant-detail'})
    return infotable

In [4]:
def ready_to_scrap_page_detail_review(url):
    _ = req.Request(url)
    HTML = req.urlopen(_)
    page = bs(HTML.read(), 'html.parser')
    reviewtable = page.find('section', {'class':'module review-container'}).findAll('li',{'class':'review_fliter_item'})
    return reviewtable

In [5]:
def page_scrap_batch(target_url):
    
    rest_list = ready_to_scrap_page_main(target_url)
    
    batch_list = list()
    for rest in rest_list:
        rest_name = rest.find('div', {'class':'info'}).find('h2',{'class':'title'}).get_text().strip()
        rest_genre = rest.find('div', {'class':'info'}).find('p',{'class':'etc'}).find('span').get_text()
        rest_viewcount = int(''.join(rest.find('div', {'class':'info'}).find('span', {'class':'view_count'}).get_text().split(',')))
        rest_reviewcount = int(''.join(rest.find('div', {'class':'info'}).find('span', {'class':'review_count'}).get_text().split(',')))
        rest_url = rest.find('div',{'class':'info'}).find('a')['href']
        batch_list.append([rest_name,
                          rest_genre,
                          rest_viewcount,
                          rest_reviewcount,
                          rest_url])
    
    
    for i, each_rest in enumerate(batch_list):
        info_list = ready_to_scrap_page_detail("####"+each_rest[4])
        
        try:
            rest_rate = float(info_list.find("strong",{"class":"rate-point "}).get_text().strip())
        except:
            rest_rate = None
            print(rest_name," 점수없음")
            
        rest_favoritecount = int(''.join(info_list.find("span",{"class":"cnt favorite"}).get_text().split(":")[1].split(",")))
        

        table_data = info_list.find("tbody").findAll("tr")
        rest_address = None
        rest_phone = None
        rest_price = None
        rest_open_start = None
        rest_open_end = None
        for datum in table_data:
            table_label = datum.find('th').get_text()
            if table_label == "주소":
                rest_address = datum.find('td').get_text()
                continue
            if table_label == "전화번호":
                rest_phone = datum.find('td').get_text()
                continue
            if table_label == "가격대":
                rest_price = datum.find('td').get_text()
                continue
            if table_label == "영업시간":
                rest_open_start = datum.find('td').get_text().split(" - ")[0]
                rest_open_end = datum.find('td').get_text().split(" - ")[1]
                continue
        batch_list[i] = batch_list[i] + [rest_rate, rest_favoritecount, rest_address, rest_phone, rest_price, rest_open_start, rest_open_end]
        
        review_data = ready_to_scrap_page_detail_review("####"+each_rest[4])
        review_score = list()
        for review in review_data[1:]:
            review_score.append(int(review.get_text().split("(")[1].replace(")","")))
        batch_list[i] = batch_list[i] + review_score
        
        gc.collect()
        
    return batch_list

In [6]:
# 스크래핑한 사이트 명은 비공개입니다.

df = pd.DataFrame(columns = ['name','genre','view_count','review_count','url','rate','favorite_count','address','phone','price','open_start','open_end','cnt_good','cnt_soso','cnt_bad'])
for _ in range(1,101):
    print("batch",_,"start")
    batch = pd.DataFrame(page_scrap_batch("####"+str(_)),
                         columns = ['name','genre','view_count','review_count','url','rate','favorite_count','address','phone','price','open_start','open_end','cnt_good','cnt_soso','cnt_bad'])
    df = df.append(batch, ignore_index = True)
    print("batch",_,"finished")
    time.sleep(10)
    
df.to_csv("gangnam_idx100.csv", encoding = "euc-kr")

batch 1 start
batch 1 finished
batch 2 start
batch 2 finished
batch 3 start
batch 3 finished
batch 4 start
batch 4 finished
batch 5 start
batch 5 finished
batch 6 start
batch 6 finished
batch 7 start
batch 7 finished
batch 8 start
batch 8 finished
batch 9 start
batch 9 finished
batch 10 start
batch 10 finished
batch 11 start
batch 11 finished
batch 12 start
batch 12 finished
batch 13 start
batch 13 finished
batch 14 start
batch 14 finished
batch 15 start
batch 15 finished
batch 16 start
batch 16 finished
batch 17 start
육덕식당  점수없음
batch 17 finished
batch 18 start
batch 18 finished
batch 19 start
batch 19 finished
batch 20 start
batch 20 finished
batch 21 start
batch 21 finished
batch 22 start
batch 22 finished
batch 23 start
전주전집  점수없음
batch 23 finished
batch 24 start
batch 24 finished
batch 25 start
봉산옥  점수없음
봉산옥  점수없음
batch 25 finished
batch 26 start
몽키동키  점수없음
몽키동키  점수없음
몽키동키  점수없음
batch 26 finished
batch 27 start
악소레커  점수없음
악소레커  점수없음
악소레커  점수없음
악소레커  점수없음
악소레커  점수없음
악소레커  점수없음
batc